## Extracting SNPs contributing to a model

In [18]:
import os
import pickle
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

In [19]:
os.chdir('/media/HDD_4TB_1/jordi/cfuses_gnn_enrollhd_2024/')

In [20]:
# Data directory
data_dir = "data/features/"

# Path to regressors
reg_dir = "data/ml_results/regressors/"

# Path to output directory
out_dir = "data/ml_results/regressors_info/"

In [21]:
# Extract feature matrix header
with open(data_dir + "subsetting/header_feature_matrix_m3_filt_0.01.txt", "r") as file:
    header = file.readline().strip().split("\t")
        
feature_names = header[1:]

# Read SNP lookup table
snp_lookuptab = pd.read_csv("data/SNPs/snps_gene_GO_m3.txt", sep="\t")

## For linear methods

In [33]:
# Name of pickled model file
regressor_file = 'elastic_net_regressor_bsc.pkl' # Elastic Net
# regressor_file = 'lasso_regressor_bsc.pkl' # Lasso

# Extract model name string
regressor_name = regressor_file.split(sep = ".")[0]

In [34]:
# Load the pickled model
with open(reg_dir + regressor_file, 'rb') as f:
    regressor = pickle.load(f)

In [35]:
# Get how many coefficients different from 0
sum(1 for coef in regressor.coef_ if coef != 0)

1164

In [36]:
# Sort coefficients in descending order by their absolute value
non_zero_indices_values = [(index, coef, abs(coef)) for index, coef in enumerate(regressor.coef_) if coef != 0]
sorted_indices_values = sorted(non_zero_indices_values, key=lambda x: x[2], reverse=True)

Get what snps correspond to each index

In [37]:
# Keep only the feature corresponding to the index and signed 
# value of the sorted coefficients
model_coefs = []

for index, coef, _ in sorted_indices_values:
    model_coefs.append([feature_names[index], coef])

In [38]:
# If most important feature is CAG (will most 
# certainly be) print its coefficient

if model_coefs[0][0] == 'CAG':
    print('CAG coefficient is:', model_coefs[0][1])

CAG coefficient is: -3.2781513


In [39]:
# Assemble presenting table
model_snps = pd.DataFrame(columns=['SNP','Coefficient','Gene','GO'])

for snp, coef in model_coefs[1:]:
    # Find snp in lookup table
    match = snp_lookuptab[snp_lookuptab["SNP"] == snp]
    # Retrieve corresponding gene and GO term
    gene, GO = match['Gene'].values[0], match['GO_term'].values[0]
    # Create new row in pd df
    row = {'SNP':snp, 'Coefficient':coef, 'Gene':gene, 'GO':GO}
    model_snps = model_snps._append(row, ignore_index=True)

/tmp/ipykernel_449942/1903723015.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_snps = model_snps._append(row, ignore_index=True)


In [30]:
model_snps

,SNP,Coefficient,Gene,GO
0,rs17173770,3.656065e-02,SMARCD3,GO:0140110
1,rs61997076,3.399666e-02,FAN1,extra_genes
2,rs144287831,2.923338e-02,MLH1,GO:0006298
3,rs10885398,-2.851474e-02,TCF7L2,GO:0140110
4,rs8017707,-2.680090e-02,ESRRB,GO:0140110
...,...,...,...,...
861,rs2071534,-4.730222e-08,PSMB9,GO:0043161
862,rs10072296,3.193814e-08,JMY,GO:0140110
863,rs894414,1.890893e-08,SMARCD2,GO:0140110
864,rs2721963,1.772747e-08,TRPS1,GO:0140110


In [40]:
# Save final table to tab separated txt file
model_snps.to_csv(out_dir + regressor_name + '_coefs.txt', sep='\t', index=False)

## For tree based methods

### XGBoost models

In [51]:
# Name of pickled model file
# xgb_booster_file = 'histXGBoost_regressor_bsc.pkl'
xgb_booster_file = 'approxXGBoost_regressor_bsc.pkl'

# Extract model name string
xgb_booster_name = xgb_booster_file.split(sep = ".")[0]

In [52]:
# Load the pickled model
with open(reg_dir + xgb_booster_file, 'rb') as f:
    xgb_booster = pickle.load(f)

In [53]:
gain_importances = xgb_booster.get_booster().get_score(importance_type='gain')

In [54]:
weight_importances = xgb_booster.get_booster().get_score(importance_type='weight')

In [55]:
# Convert gain_importances dictionary to a list of lists
# removing the intial f of the key
gain_importances_list = [[feature_names[int(key[1:])], value] for key, value in gain_importances.items()]

In [56]:
# Sort in descending order
gain_importances_list = sorted(gain_importances_list, key=lambda x: x[1], reverse=True)

In [57]:
# If most important feature is CAG (will most 
# certainly be) print its coefficient

if gain_importances_list[0][0] == 'CAG':
    print('CAG coefficient is:', gain_importances_list[0][1])

CAG coefficient is: 209.2954559326172


In [58]:
# Assemble presenting table
booster_important_snps = pd.DataFrame(columns=['SNP','Gain','Gene','GO'])

# Take top 100 snps
for snp, gain in gain_importances_list[1:101]:
    # Find snp in lookup table
    match = snp_lookuptab[snp_lookuptab["SNP"] == snp]
    # Retrieve corresponding gene and GO term
    gene, GO = match['Gene'].values[0], match['GO_term'].values[0]
    # Create new row in pd df
    row = {'SNP':snp, 'Gain':gain, 'Gene':gene, 'GO':GO}
    booster_important_snps = booster_important_snps._append(row, ignore_index=True)

/tmp/ipykernel_449942/1342472698.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  booster_important_snps = booster_important_snps._append(row, ignore_index=True)


In [59]:
# Save final table to tab separated txt file
booster_important_snps.to_csv(out_dir + xgb_booster_name + '_coefs.txt', sep='\t', index=False)

----

## Random Forest

In [60]:
# Name of pickled model file
rf_file = 'randomforest_regressor.pkl'

In [61]:
# Extract model name string
rf_name = rf_file.split(sep = ".")[0]
# Load the pickled model
with open(reg_dir + rf_file, 'rb') as f:
    rf_reg = pickle.load(f)

In [63]:
# Get feature importances (gini importance)
feat_importances = rf_reg.feature_importances_

In [65]:
# Sort ginis importance in descending order
non_zero_indices_values = [(index, coef) for index, coef in enumerate(feat_importances) if coef != 0]
sorted_indices_values = sorted(non_zero_indices_values, key=lambda x: x[1], reverse=True)

In [66]:
# Keep only the feature corresponding to the index and signed 
# value of the sorted coefficients
model_ginis = []

for index, coef in sorted_indices_values:
    model_ginis.append([feature_names[index], coef])

In [67]:
# If most important feature is CAG (will most 
# probably be) print its coefficient

if model_ginis[0][0] == 'CAG':
    print('CAG coefficient is:', model_ginis[0][1])

CAG coefficient is: 0.8760109982060723


In [74]:
# Assemble presenting table
model_snps = pd.DataFrame(columns=['SNP','Gini_Importance','Gene','GO'])

for snp, gini in model_ginis[1:]:
    # Find snp in lookup table
    match = snp_lookuptab[snp_lookuptab["SNP"] == snp]
    # Retrieve corresponding gene and GO term
    gene, GO = match['Gene'].values[0], match['GO_term'].values[0]
    # Create new row in pd df
    row = {'SNP':snp, 'Gini_Importance':gini, 'Gene':gene, 'GO':GO}
    model_snps = model_snps._append(row, ignore_index=True)

/tmp/ipykernel_449942/3332031514.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_snps = model_snps._append(row, ignore_index=True)


In [76]:
# Save final table to tab separated txt file
model_snps.to_csv(out_dir + rf_name + '_coefs.txt', sep='\t', index=False)

In [75]:
model_snps

,SNP,Gini_Importance,Gene,GO
0,rs77752857,0.001354,NEK6,GO:0031625
1,rs1043742,0.001183,CUL2,GO:0043161
2,rs11201880,0.000847,GRID1,GO:0035249
3,rs118089305,0.000672,FAN1,extra_genes
4,rs17767868,0.000665,ZFAT,GO:0140110
...,...,...,...,...
879,rs72885925,0.000051,TRIM62,GO:0140110
880,rs4322226,0.000051,PIGM,GO:0042157
881,rs10246722,0.000050,HDAC9,GO:0140110
882,rs4802994,0.000050,ZNF600,GO:0140110
